Lab2
Тоог нэмэх параллел алгоритм



In [ ]:
%%writefile parallel_sum.cpp
#include<iostream>
#include<omp.h>
#include<chrono>
using namespace std;

#define ARRAY_SIZE 1000000

int parallel(int A[]) {
    int total_sum = 0;

    #pragma omp parallel num_threads(4)
    {
        int l_sum = 0;
        #pragma omp for
        for (int i = 0; i < ARRAY_SIZE; ++i) {
            l_sum += A[i];
        }
        //#pragma omp critical
        total_sum += l_sum;
    }
    return total_sum;
}

int arraySum(int arr[]) {
    int sum = 0;
    for (int i = 0; i < ARRAY_SIZE; ++i) {
        sum += arr[i];
    }
    return sum;
}

int main() {
    int A[ARRAY_SIZE];

    // Populate array A with values
    for (int i = 0; i < ARRAY_SIZE; ++i) {
        A[i] = i;
    }

    auto start_time_serial = chrono::high_resolution_clock::now();

    int result_serial = arraySum(A);

    auto end_time_serial = chrono::high_resolution_clock::now();
    chrono::duration<double> serial_duration = end_time_serial - start_time_serial;

    auto start_time_parallel = chrono::high_resolution_clock::now();
    int parallel_res = parallel(A);
    auto end_time_parallel = chrono::high_resolution_clock::now();
    chrono::duration<double> parallel_duration = end_time_parallel - start_time_parallel;

    cout << "Total sum of array elements (serial): " << result_serial << endl;
    cout << "Total sum of array elements (parallel): " << parallel_res << endl;

    cout << "Serial time: " << serial_duration.count() << " seconds" << endl;
    cout << "Parallel time: " << parallel_duration.count() << " seconds" << endl;
    cout << "Speedup: " << serial_duration.count() / parallel_duration.count() << endl;

    return 0;
}

Overwriting parallel_sum.cpp


In [ ]:
!g++ -o parallel_sum parallel_sum.cpp -fopenmp
!./parallel_sum

Total sum of array elements (serial): 1783293664
Total sum of array elements (parallel): 1783293664
Serial time: 0.00221937 seconds
Parallel time: 0.00179053 seconds
Speedup: 1.23951


Jacobi iteration алгоритм

In [ ]:
!pip install mpi4py

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import time
from mpi4py import MPI

# MPI эхлүүлж, MPI коммуникаторын зэрэглэл болон хэмжээг олж авна.
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

start = time.time()

MAX_N = 8

A = np.zeros((MAX_N-2, MAX_N-2))
A = np.pad(A, pad_width=1, mode='constant', constant_values=1)
(row_num, col_num) = A.shape
B = np.zeros((row_num, col_num))

#Процесс бүрт хуваарилагдсан мөрийн тоог тодорхойлно. Нийт мөрийн тоог процессын тоонд хуваахгүй бол сүүлийн процесс нэмэлт мөр авч болно.
if rank == (size-1):
    local_row_num = int(MAX_N / size) + (MAX_N % size)
else:
    local_row_num = int(MAX_N / size)

#A` болон B матрицуудын хэсгүүдийг зэрэглэлд үндэслэн процесс бүрт хуваарилдаг.
local_first_row = rank * int(MAX_N / size)
local_last_row = local_first_row + local_row_num
A_local = A[local_first_row:local_last_row,]
B_local = B[local_first_row:local_last_row,]

converge = False
iteration_num = 0
while not converge:  #Нэгдмэл байдалд хүрэх хүртэл Якоби давталтыг гүйцэтгэнэ.
    iteration_num += 1
    diffnorm = 0.0

    A_local_padding = np.pad(A_local, pad_width=1, mode='constant', constant_values=0)

#MPI Send болон Recv үйлдлүүдийг ашиглан хөрш процессуудын хооронд `A_local`-ын  утгуудыг холбодог.
    if rank > 0:
        comm.Send(A_local[0, ], dest=rank-1, tag=11)
        tmp = np.empty(MAX_N, dtype=A_local_padding.dtype)
        comm.Recv(tmp, source=rank-1, tag=22)
        A_local_padding[0, 1:MAX_N+1] = tmp
    if rank < (size - 1):
        comm.Send(A_local[local_row_num-1, ], dest=rank+1, tag=22)
        tmp = np.empty(MAX_N, dtype=A_local_padding.dtype)
        comm.Recv(tmp, source=rank+1, tag=11)
        A_local_padding[local_row_num+1, 1:MAX_N+1] = tmp

    for i in range(row_num):
        for j in range(col_num):
            idx_i_A = i + 1
            idx_j_A = j + 1
            B_local[i][j] = 0.25*(A_local_padding[idx_i_A+1, idx_j_A]
                                + A_local_padding[idx_i_A-1, idx_j_A]
                                + A_local_padding[idx_i_A, idx_j_A+1]
                                + A_local_padding[idx_i_A, idx_j_A-1])
            diffnorm += math.sqrt((B_local[i, j] - A_local[i, j])*(B_local[i, j] - A_local[i, j]))
    A_local = np.copy(B_local)

    diffnorm_glov = comm.allreduce(diffnorm, op=MPI.SUM)
# нийлмэл байдлыг шалгана.
    if diffnorm_glov <= 0.0001:
        print('Converge, iteration : %d per process' % iteration_num)
        print('Error : %f' % diffnorm_glov)
        converge = True
#Бүх процессоос 'A' матрицын локал хэсгүүдийг эх процесс руу (0-р зэрэглэл) цуглуулдаг.
comm.Gatherv(A_local, [A, MPI.DOUBLE], root=0)

if rank == 0:
    end = time.time()
    parallel_time = end - start
    print('Parallel Time:', parallel_time, 'seconds')

if rank == 0:
    start = time.time()
    end = time.time()
    serial_time = end - start
    print('Serial Time:', serial_time, 'seconds')

if rank == 0:
    speedup = serial_time / parallel_time
    print('Speedup:', speedup)

Converge, iteration : 144 per process
Error : 0.000096
Parallel Time: 0.057433128356933594 seconds
Serial Time: 2.384185791015625e-07 seconds
Speedup: 4.151237899141524e-06


In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import time
from mpi4py import MPI

# Function to print the result
def print_result(A, rank):
    if rank == 0:
        print("Result Matrix A:")
        print(A)

# Main block
if __name__ == "__main__":
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    start = time.time()

    MAX_N = 8

    A = np.zeros((MAX_N-2, MAX_N-2))
    A = np.pad(A, pad_width=1, mode='constant', constant_values=1)
    (row_num, col_num) = A.shape
    B = np.zeros((row_num, col_num))

    if rank == (size-1):
        local_row_num = int(MAX_N / size) + (MAX_N % size)
    else:
        local_row_num = int(MAX_N / size)

    local_first_row = rank * int(MAX_N / size)
    local_last_row = local_first_row + local_row_num
    A_local = A[local_first_row:local_last_row, ]
    B_local = B[local_first_row:local_last_row, ]

    converge = False
    iteration_num = 0
    while not converge:
        iteration_num += 1
        diffnorm = 0.0

        A_local_padding = np.pad(A_local, pad_width=1, mode='constant', constant_values=0)

        if rank > 0:
            comm.Send(A_local[0, ], dest=rank-1, tag=11)
            tmp = np.empty(MAX_N, dtype=A_local_padding.dtype)
            comm.Recv(tmp, source=rank-1, tag=22)
            A_local_padding[0, 1:MAX_N+1] = tmp
        if rank < (size - 1):
            comm.Send(A_local[local_row_num-1, ], dest=rank+1, tag=22)
            tmp = np.empty(MAX_N, dtype=A_local_padding.dtype)
            comm.Recv(tmp, source=rank+1, tag=11)
            A_local_padding[local_row_num+1, 1:MAX_N+1] = tmp

        for i in range(row_num):
            for j in range(col_num):
                idx_i_A = i + 1
                idx_j_A = j + 1
                B_local[i][j] = 0.25*(A_local_padding[idx_i_A+1, idx_j_A]
                                    + A_local_padding[idx_i_A-1, idx_j_A]
                                    + A_local_padding[idx_i_A, idx_j_A+1]
                                    + A_local_padding[idx_i_A, idx_j_A-1])
                diffnorm += math.sqrt((B_local[i, j] - A_local[i, j])*(B_local[i, j] - A_local[i, j]))
        A_local = np.copy(B_local)

        diffnorm_glov = comm.allreduce(diffnorm, op=MPI.SUM)
        if diffnorm_glov <= 0.0001:
            print('Converge, iteration : %d per process' % iteration_num)
            print('Error : %f' % diffnorm_glov)
            converge = True

    comm.Gatherv(A_local, [A, MPI.DOUBLE], root=0)


print_result(A, rank)

Converge, iteration : 144 per process
Error : 0.000096
Result Matrix A:
[[5.42518598e-06 1.01960145e-05 1.37370531e-05 1.56212004e-05
  1.56212004e-05 1.37370531e-05 1.01960145e-05 5.42518598e-06]
 [1.01960145e-05 1.91622391e-05 2.58172149e-05 2.93582536e-05
  2.93582536e-05 2.58172149e-05 1.91622391e-05 1.01960145e-05]
 [1.37370531e-05 2.58172149e-05 3.47834396e-05 3.95542680e-05
  3.95542680e-05 3.47834396e-05 2.58172149e-05 1.37370531e-05]
 [1.56212004e-05 2.93582536e-05 3.95542680e-05 4.49794540e-05
  4.49794540e-05 3.95542680e-05 2.93582536e-05 1.56212004e-05]
 [1.56212004e-05 2.93582536e-05 3.95542680e-05 4.49794540e-05
  4.49794540e-05 3.95542680e-05 2.93582536e-05 1.56212004e-05]
 [1.37370531e-05 2.58172149e-05 3.47834396e-05 3.95542680e-05
  3.95542680e-05 3.47834396e-05 2.58172149e-05 1.37370531e-05]
 [1.01960145e-05 1.91622391e-05 2.58172149e-05 2.93582536e-05
  2.93582536e-05 2.58172149e-05 1.91622391e-05 1.01960145e-05]
 [5.42518598e-06 1.01960145e-05 1.37370531e-05 1.5621